In [1]:
import os
import re
import csv
import base64
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from bs4 import BeautifulSoup
from wordcloud import WordCloud

In [2]:
url1 = 'https://liquipedia.net/apexlegends/Portal:Teams/Americas'
url2 = 'https://liquipedia.net/apexlegends/Portal:Teams/EMEA'
url3 = 'https://liquipedia.net/apexlegends/Portal:Teams/APAC_North'
url4 = 'https://liquipedia.net/apexlegends/Portal:Teams/APAC_South'
url5 = 'https://liquipedia.net/apexlegends/Portal:Players'
Headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36 Edg/113.0.1774.50','Referer': 'https://liquipedia.net/apexlegends/Portal:Teams','Accept-Language':'zh-CN,zh;q=0.9,en;q=0.8,en-GB;q=0.7,en-US;q=0.6',
'Cookie':'_pk_id.1.4442=59e8dd207cfb54ed.1684816505.; _pbjs_userid_consent_data=3524755945110770; na-unifiedid={"TDID":"b1ac3fcf-c7ec-4157-bb47-a7aaed02cbba","TDID_LOOKUP":"FALSE","TDID_CREATED_AT":"2023-05-23T04:35:14"}; cto_bundle=xIvh9l9OaW1wQSUyQiUyQkxoM3Jvdng3S2NpWXd6ejBMWjh1bkEzdDdEV0tORlM0djJDdHR5em9OSkt2VlMlMkY0TW96cGVTWG9NWWpMJTJGY2xtR2l6NXBuRU5MQ3Z2VE5iNE4ycWlacUxXTmZXTHVRMXhaY0szbGpMSG0wdGtVJTJGandQNkJpdEViRUI; _pk_ref.1.4442=["","",1684826270,"https://www.bing.com/"]; _ga_3TE8DNE0DL=GS1.1.1684826270.2.0.1684826270.0.0.0; _ga=GA1.2.1380062086.1684816506'
}
response1 = requests.get(url=url1,headers=Headers)
response2 = requests.get(url=url2,headers=Headers)
response3 = requests.get(url=url3,headers=Headers)
response4 = requests.get(url=url4,headers=Headers)
response5 = requests.get(url=url5,headers=Headers)

In [ ]:
#爬取全球各赛区战队名称
def getTeam(html):
    team_list = []
    soup = BeautifulSoup(html,'html.parser')
    team_list.append((soup.find('li',{'class':'active'})).find('a').string)
    div = soup.find_all('span',{'class':'team-template-text'})

    for li in div:
        a = li.find('a')
        text = a.string
        team_list.append(text)
        
    return team_list
    

In [ ]:
#爬取各战队所拥有队员ID
def getplayerID(html):
    Id = []
    soup = BeautifulSoup(html,'html.parser')
    div = soup.find_all('span',style="white-space:pre")

    for li in div:
        a = li.find('a')
        text = a.string
        Id.append(text)

    return Id
    

In [ ]:
#爬取各战队所拥有队员真实姓名
def getplayerName(html):
    Real_Name = []
    a = re.findall(r'<td>(.*?)</td>', html, re.I|re.M)

    for li in a:
        if li == "":
            li = "UNKNOWN"
        text = li
        Real_Name.append(text)

    return Real_Name


In [ ]:
#爬取全球各赛区战队及其队员的信息
csvf = open('Apex Legends Global Teams and Players.csv','w',encoding='utf-8')
fw = csv.writer(csvf)
fw.writerow(getTeam(response1.text))
fw.writerow(getplayerID(response1.text))
fw.writerow(getplayerName(response1.text))
fw.writerow(getTeam(response2.text))
fw.writerow(getplayerID(response2.text))
fw.writerow(getplayerName(response2.text))
fw.writerow(getTeam(response3.text))
fw.writerow(getplayerID(response3.text))
fw.writerow(getplayerName(response3.text))
fw.writerow(getTeam(response4.text))
fw.writerow(getplayerID(response4.text))
fw.writerow(getplayerName(response4.text))
csvf.close()

In [3]:
#爬取个人价值排名前20的选手
def getHighlightedPlayer(html):
    HighlightedPlayer_list = []
    soup = BeautifulSoup(html,'html.parser')
    div = soup.find_all('tr',style="line-height:25px")
    line = re.findall(r'<td><a[^>]*>(.*?)</a></td>', html, re.I|re.M)

    for li in line:
        HighlightedPlayer_list.append(li)

    return(HighlightedPlayer_list)    


In [4]:
#爬取高价值选手的个人信息
def getPlayerInf(html):
    HighlightedPlayer = getHighlightedPlayer(html)
    Information = []
    for i in range(20):
        HighlightedPlayer[i] = HighlightedPlayer[i].replace(' ','_')
        url = 'https://liquipedia.net/apexlegends/'+HighlightedPlayer[i]
        res = requests.get(url=url,headers=Headers)
        soup = BeautifulSoup(res.text,'html.parser')
        div1 = soup.find_all('p')
        a = div1[0].text
        Information.append(a)
        
    return Information  


In [8]:
def getPlayerInf2(html):
    HighlightedPlayer = getHighlightedPlayer(html)
    Rank = []
    for i in range(20):
        HighlightedPlayer[i] = HighlightedPlayer[i].replace(' ','_')
        url = 'https://liquipedia.net/apexlegends/'+HighlightedPlayer[i]
        res = requests.get(url=url,headers=Headers)
        soup = BeautifulSoup(res.text,'html.parser') 
        div2 = soup.find_all('b',{'class':"placement-text"})
        for li in div2:
            text = li.string
            Rank.append(text)    
        
    return Rank

In [6]:
def getPlayerInf3(html):
    HighlightedPlayer = getHighlightedPlayer(html)
    Match = []
    for i in range(20):
        HighlightedPlayer[i] = HighlightedPlayer[i].replace(' ','_')
        url = 'https://liquipedia.net/apexlegends/'+HighlightedPlayer[i]
        res = requests.get(url=url,headers=Headers)
        soup = BeautifulSoup(res.text,'html.parser') 
        div3 = soup.find_all('td',style="text-align:left")
        for li in div3:
            a = li.find('a')
            text = a
            Match.append(text)     
        
    return Match


In [7]:
def getPlayerInf4(html):
    HighlightedPlayer = getHighlightedPlayer(html)
    Date = []
    for i in range(20):
        HighlightedPlayer[i] = HighlightedPlayer[i].replace(' ','_')
        url = 'https://liquipedia.net/apexlegends/'+HighlightedPlayer[i]
        res = requests.get(url=url,headers=Headers)
        soup = BeautifulSoup(res.text,'html.parser') 
        div3 = soup.find('div',{'class':"table-responsive"}).find('tbody').find_all('tr')
        for li in div3:
            a = li.find('td')
            text = a
            Date.append(text)

    return Date

In [9]:
#爬取个人价值排名前20的选手的个人信息
csvf = open('Top 20 Players in Apex Legends .csv','w',encoding='utf-8')
fw = csv.writer(csvf)
fw.writerow(getHighlightedPlayer(response5.text))
fw.writerow(getPlayerInf(response5.text))
fw.writerow(getPlayerInf4(response5.text))
fw.writerow(getPlayerInf2(response5.text))
fw.writerow(getPlayerInf3(response5.text))
csvf.close()

In [14]:
def getsrc(html):
    HighlightedPlayer = getHighlightedPlayer(html)
    src = []
    for i in range(20):
        HighlightedPlayer[i] = HighlightedPlayer[i].replace(' ','_')
        url = 'https://liquipedia.net/apexlegends/'+HighlightedPlayer[i]
        res = requests.get(url=url,headers=Headers)
        soup = BeautifulSoup(res.text,'html.parser')
        divp = soup.find('a',{'class':'image'}).find('img').get('src')
        src.append(divp)
    return src
print(getsrc(response5.text))

['/commons/images/thumb/1/1e/TSM_ImperialHal_at_the_ALGS_Stockholm_Playoffs.jpg/600px-TSM_ImperialHal_at_the_ALGS_Stockholm_Playoffs.jpg', '/commons/images/thumb/d/d2/TSM_Verhulst_at_the_ALGS_Stockholm_Playoffs.jpg/600px-TSM_Verhulst_at_the_ALGS_Stockholm_Playoffs.jpg', '/commons/images/thumb/b/bb/TSM_Reps_at_the_ALGS_Raleigh_Championship.jpg/600px-TSM_Reps_at_the_ALGS_Raleigh_Championship.jpg', '/commons/images/thumb/5/56/NRG_sweetdreams_at_the_ALGS_London_Split_1_Playoffs.jpg/600px-NRG_sweetdreams_at_the_ALGS_London_Split_1_Playoffs.jpg', '/commons/images/thumb/c/c2/NRG_Gild_at_the_ALGS_London_Split_1_Playoffs.jpg/600px-NRG_Gild_at_the_ALGS_London_Split_1_Playoffs.jpg', '/commons/images/thumb/7/78/NRG_nafen_at_the_ALGS_London_Split_1_Playoffs.jpg/600px-NRG_nafen_at_the_ALGS_London_Split_1_Playoffs.jpg', '/commons/images/thumb/1/1d/Acend_PostKiLL_at_the_ALGS_2022_Championship.jpg/600px-Acend_PostKiLL_at_the_ALGS_2022_Championship.jpg', '/commons/images/thumb/f/fc/Acend_Lufka_at_the_AL

In [21]:
#爬取个人价值前20名选手的照片
def getPictures(url,path):
    res = requests.get(url=url,headers=Headers)
    with open(path,'wb')as f:
        for chunk in res.iter_content(chunk_size=128):
            f.write(chunk)

def getTop20PlayerPictures(html):
    src_list = getsrc(html)
    HighlightedPlayer = getHighlightedPlayer(html)
    for i in range(20):
        urlp = 'https://liquipedia.net'+src_list[i]
        path = HighlightedPlayer[i]+'.jpg'
        getPictures(urlp,path)

In [22]:
getTop20PlayerPictures(response5.text)

In [3]:
#爬取2019-2023年Top50的战队及队员的近年数据
def getTeamData():
    TeamData = []
    for i in range(2019,2024):
        urldata = 'https://liquipedia.net/apexlegends/Portal:Statistics/{}'.format(i)
        res = requests.get(url=urldata,headers=Headers)
        soup = BeautifulSoup(res.text,'html.parser')
        divt = soup.find_all('div',{'class':"divRow"})
        
        for li in divt:
            team = li.find('span',{'class':"team-template-text"}).find('a')
            _1st = li.find_all('div',{'class':"divCell"})[2].text 
            _2nd = li.find_all('div',{'class':"divCell"})[3].text
            _3rd_4th = li.find_all('div',{'class':"divCell"})[4].text
            Totalwinnings = li.find_all('div',{'class':"divCell"})[5].text
            TeamData.append(team.string+" owns ["+_1st+" 1st] ["+_2nd+" 2nd] ["+_3rd_4th+" 3rd/4th] ["+Totalwinnings+" Totalwinnings]") 
    
    return TeamData

def getPlayerData():
    PlayerData = []
    for i in range(2019,2024):
        urldata = 'https://liquipedia.net/apexlegends/Portal:Statistics/{}'.format(i)
        res = requests.get(url=urldata,headers=Headers)
        soup = BeautifulSoup(res.text,'html.parser')
        divp = soup.find_all('tr',style="line-height:25px")

        for li in divp:
            player = li.find_all('td')[1].find('a')
            _1st = li.find_all('td',style="text-align:center")[0].text 
            _2nd = li.find_all('td',style="text-align:center")[1].text
            _3rd_4th = li.find_all('td',style="text-align:center")[2].text
            _S_Tier = li.find_all('td',style="text-align:center")[3].text
            Earnings = li.find_all('td')[6].string
            PlayerData.append(player.string+" owns ["+_1st+" 1st] ["+_2nd+" 2nd] ["+_3rd_4th+" 3rd/4th] ["+_S_Tier+" S-Tier] ["+Earnings+" Earnings]")    
    
    return PlayerData

In [5]:
csvf = open('2019-2023 Team Statics .csv','w',encoding='utf-8')
fw = csv.writer(csvf)
fw.writerow(getTeamData())
csvf.close()

csvf = open('2019-2023 Player Statics .csv','w',encoding='utf-8')
fw = csv.writer(csvf)
fw.writerow(getPlayerData())
csvf.close()

In [89]:
#爬取所有队伍与选手信息
def getTeamOverall():
    TeamOverall = []
    urldata = 'https://liquipedia.net/apexlegends/Portal:Statistics/Team_earnings'
    res = requests.get(url=urldata,headers=Headers)
    soup = BeautifulSoup(res.text,'html.parser')
    divt = soup.find('div',{'class':"mw-parser-output"}).find('tbody').find_all('tr')
    i = 0    

    for li in divt:
        if i != 0:
            team = li.find('span',{'class':"team-template-text"}).find('a').string
            _1st = li.find_all('td',style="text-align:center")[0].text 
            _2nd = li.find_all('td',style="text-align:center")[1].text 
            _3rd_4th = li.find_all('td',style="text-align:center")[2].text 
            _S_Tier = li.find_all('td',style="text-align:center")[3].text
            Totalwinnings = li.find_all('td')[5].text
            TeamOverall.append(team+" owns ["+_1st+" 1st] ["+_2nd+" 2nd] ["+_3rd_4th+" 3rd/4th] ["+_S_Tier+" S-Tier] ["+Totalwinnings+" Totalwinnings]") 
        i += 1
  
    return TeamOverall

In [90]:
def getPlayerOverall():
    PlayerOverall = []
    for i in range(2019,2024):
        urldata = 'https://liquipedia.net/apexlegends/Portal:Statistics/Player_earnings'
        res = requests.get(url=urldata,headers=Headers)
        soup = BeautifulSoup(res.text,'html.parser')
        divp = soup.find_all('tr',style="line-height:25px")

        for li in divp:
            player = li.find_all('td')[1].find('a')
            _1st = li.find_all('td',style="text-align:center")[0].text 
            _2nd = li.find_all('td',style="text-align:center")[1].text
            _3rd_4th = li.find_all('td',style="text-align:center")[2].text
            _S_Tier = li.find_all('td',style="text-align:center")[3].text
            Earnings = li.find_all('td')[6].string
            PlayerOverall.append(player.string+" owns ["+_1st+" 1st] ["+_2nd+" 2nd] ["+_3rd_4th+" 3rd/4th] ["+_S_Tier+" S-Tier] ["+Earnings+" Earnings]")    
    
    return PlayerOverall


In [91]:
#爬取最年轻以及最年长的选手
def getAge():
    Youngest = []
    Oldest = []
    urldata = 'https://liquipedia.net/apexlegends/Portal:Statistics'
    res = requests.get(url=urldata,headers=Headers)
    soup = BeautifulSoup(res.text,'html.parser')
    divy = soup.find_all('table',{'class':"wikitable smwtable jquery-tablesorter sortable"},style="margin-left:0px;margin-right:auto")
    i = 0
    for li2 in divy:
        divo = li2.find('tbody').find_all('tr')
        for li in divo:
            if i != 0  and i <= 20:
                id = li.find('b').find('a').string
                age = li.find_all('td')[2].text
                Youngest.append(id + " " + age)
            elif i > 20 and i != 21:
                id = li.find('b').find('a').string
                age = li.find_all('td')[2].text
                Oldest.append(id + " " + age)
            i += 1
    return Youngest,Oldest

In [92]:
csvf = open('TeamOverall.csv','w',encoding='utf-8')
fw = csv.writer(csvf)
fw.writerow(getTeamOverall())
csvf.close()

csvf = open('PlayerOverall.csv','w',encoding='utf-8')
fw = csv.writer(csvf)
fw.writerow(getPlayerOverall())
csvf.close()

csvf = open('PlayerAge.csv','w',encoding='utf-8')
fw = csv.writer(csvf)
fw.writerow(getAge())
csvf.close()